In [1]:
%pip install sqlalchemy pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import glob

glob.glob('../data/raw/StationFootfall*.csv')

['../data/raw\\StationFootfall_2021.csv',
 '../data/raw\\StationFootfall_2022.csv',
 '../data/raw\\StationFootfall_2023.csv',
 '../data/raw\\StationFootfall_2024_2025.csv']

In [3]:
import pandas as pd

all_dfs = []

# use glob method to retreive all files and append to list
for one_filename in glob.glob('../data/raw/StationFootfall*.csv'):
    print(f'Loading file {one_filename}')
    new_df = pd.read_csv(one_filename)
    all_dfs.append(new_df)

len(all_dfs)

Loading file ../data/raw\StationFootfall_2021.csv
Loading file ../data/raw\StationFootfall_2022.csv
Loading file ../data/raw\StationFootfall_2023.csv
Loading file ../data/raw\StationFootfall_2024_2025.csv


4

In [4]:
df = pd.concat(all_dfs)
df.shape
df.head()

,TravelDate,DayOFWeek,Station,EntryTapCount,ExitTapCount,DayOfWeek
0,20210101,Friday,Abbey Road DLR,82,128,NaN
1,20210101,Friday,Abbey Wood,0,34,NaN
2,20210101,Friday,Acton Central,221,289,NaN
3,20210101,Friday,Acton Main Line,43,53,NaN
4,20210101,Friday,Acton Town,694,791,NaN


In [9]:
from pandas.api.types import CategoricalDtype


# only rename if old columns still exist
if 'TravelDate' in df.columns:
    df.rename(columns={'TravelDate': 'Date'}, inplace=True)

if 'DayOfWeek' in df.columns:
    # if we have both, combine, if not, just ensure name is right
    if 'DayOFWeek' in df.columns:
        df['DayOFWeek'] = df['DayOFWeek'].combine_first(df['DayOfWeek'])
        df.drop(columns=['DayOfWeek'], inplace=True)
    else:
        df.rename(columns={'DayOfWeek': 'DayOFWeek'}, inplace=True)

# clean up strings and set the categorical type
cats = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
cat_type = CategoricalDtype(categories=cats, ordered=True)

df['DayOFWeek'] = df['DayOFWeek'].astype(str).str.strip().str.title()
df['DayOFWeek'] = df['DayOFWeek'].astype(cat_type)

# convert Date to actual datetime objects for better plotting
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

# check
print("Cleaned Columns:", df.columns.tolist())
print(df.head())

missing_per_column = df.isna().sum()
print('\nMissing values:')
print(missing_per_column)

print(f"\nDataset starts at: {df['Date'].min()}")
print(f"Dataset ends at: {df['Date'].max()}\n")

df.dtypes

Cleaned Columns: ['Date', 'DayOFWeek', 'Station', 'EntryTapCount', 'ExitTapCount']
        Date DayOFWeek          Station  EntryTapCount  ExitTapCount
0 1970-01-01    Friday   Abbey Road DLR             82           128
1 1970-01-01    Friday       Abbey Wood              0            34
2 1970-01-01    Friday    Acton Central            221           289
3 1970-01-01    Friday  Acton Main Line             43            53
4 1970-01-01    Friday       Acton Town            694           791

Missing values:
Date             0
DayOFWeek        0
Station          0
EntryTapCount    0
ExitTapCount     0
dtype: int64

Dataset starts at: 1970-01-01 00:00:00
Dataset ends at: 1970-01-01 00:00:00



Date             datetime64[ns]
DayOFWeek              category
Station                  object
EntryTapCount             int64
ExitTapCount              int64
dtype: object

In [6]:
import sys
sys.path.append('..') # go up to root to find src
from src.load_to_sql import upload_dataframe

# convert to string to ensure sqlite stores it as readable string
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
upload_dataframe(df, table_name='footfall')

Connecting to database at: c:\Users\lewis\Documents\projects\weather-tfl-footfall\data\processed\transport_weather.db
Table 'footfall' uploaded.
Current tables in DB: ['footfall', 'weather']
